![ros_control_logo](images/logos/header_logos.png)

![](images/logos/image_ROSDevDay21.png)

# From ros_control to ros2_control #

A brief, hands-on introduction to ros_control.
We will use ROS Noetic to show you
* when and how to use ros_control,
* how to implement a Hardware Abstraction Layer (HAL),
* how to implement a controller for your ros_control setup.

Building on your newfound ros_control knowledge, we move to ROS2 Foxy and

* introduce you to the concepts of ros2_control,
* show you how to implement a System component (sub-HAL),
* demonstrate how to implement a controller for a ros2_control setup.

Buckle up, there's a lot to learn today!

We are going to use a 2 degrees of freedom robot arm for demoing. 
![rrbot](images/rrbot.png)

But here's a *small* list of robots that use ros_control
![ROS1 ros_control robots](images/ros_control_montage.jpg)

## WHO ARE WE

### Bence Magyar, PhD in Robotics, Lead Software Engineer at FiveAI <div style="width: 100px;">![bence_image](images/bence.jpeg)</div> 
maintainer at ros_control and ros2_control

### ‪Denis Štogl, PhD in Robotics, Robotics Consultant at Stogl Robotics <div style="width: 100px;">![denis_image](images/denis.jpeg)</div> 
maintainer at ros2_control


## ros_control in a nutshell

![](images/overview.png)

* RobotHW is also referred to as the Hardware Abstraction Layer (HAL).
* Controllers claim resources from controller manager (safe)
* Controllers write set commands on claimed joint interface handles


![interfaces_and_controllers](images/roscontrol_interfaces_controllers.png)

Today we'll explain you how to write a RobotHW and a controller and what steps are important to pay attention to.

For the first set of demos we are going to use the ROS Noetic environment. Make sure to source first.

```
cd catkin_ws && source /opt/ros/noetic/setup.bash
catkin_make 
```


```
source catkin_ws/devel/setup.bash
```

Let's take a look at the robot we'll use!
```
roslaunch rrbot_description view_robot.launch
```

![rrbot rviz](images/rrbot_gui.png)

## Implementing a ros_control RobotHW 

The general structure of RobotHW is as follows:
* load URDF
* establish controller_manager service interfaces
* main loop
 * read()
 * update() -> call update() of controllers
 * write()

Let's write one!

```C++
class RRBotHardwareInterface : public hardware_interface::RobotHW
{
public:
  bool init(ros::NodeHandle & root_nh, ros::NodeHandle & robot_hw_nh);

  bool read(const ros::Time time, const ros::Duration period);

  bool write(const ros::Time time, const ros::Duration period);

private:
  hardware_interface::JointStateInterface joint_state_interface_;
  hardware_interface::PositionJointInterface position_command_interface_;

  std::vector<double> hw_position_commands_;
  std::vector<double> hw_position_states_;
  std::vector<double> hw_velocity_states_;
  std::vector<double> hw_effort_states_;

  std::vector<std::string> joint_names_;
};
```

while in the cpp file...

```C++
bool RRBotHardwareInterface::init(ros::NodeHandle & /*root_nh*/, ros::NodeHandle & robot_hw_nh)
{
  // Read parameters
  if (!robot_hw_nh.getParam("joint_names", joint_names_))
  {
    ROS_ERROR("Cannot find required parameter 'joint_names' on the parameter server.");
    throw std::runtime_error("Cannot find required parameter "
    "'joint_names' on the parameter server.");
  }
    
  ...

  // Initialize variables
  hw_position_states_.resize(num_joints, std::numeric_limits<double>::quiet_NaN());
  hw_position_commands_.resize(num_joints, std::numeric_limits<double>::quiet_NaN());
  hw_velocity_states_.resize(num_joints, std::numeric_limits<double>::quiet_NaN());
  hw_effort_states_.resize(num_joints, std::numeric_limits<double>::quiet_NaN());

  return true;
}

bool RRBotHardwareInterface::read(
  const ros::Time time, const ros::Duration period)
{
  // read robot states from hardware, in this example print only
  ROS_INFO_NAMED("RRBotHardwareInterface", "Reading...");

  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_position_states_.size(); ++i) {
    ROS_INFO_NAMED("RRBotHardwareInterface",
                   "Got state %.2f for joint %zu!", hw_position_states_[i], i);
  }

  return true;
}

bool RRBotHardwareInterface::write(const ros::Time time, const ros::Duration period)
{
  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_position_commands_.size(); ++i) {
    hw_position_states_[i] = hw_position_states_[i] +
                             (hw_position_commands_[i] - hw_position_states_[i]) / 100.0;
  }

  return true;
}

```

Finally, let's add a node where the RobotHW is initialized:

```C++
int main(int argc, char** argv)
{
  ros::init(argc, argv, "rrbot_hardware_interface");

  // Initialize node handles
  ...
  // Set up RobotHW and controller_manager      
  rrbot_hardware_interface::RRBotHardwareInterface rrbot_hardware_interface;
  controller_manager::ControllerManager controller_manager(&rrbot_hardware_interface, root_nh);

  rrbot_hardware_interface.init(root_nh, robot_nh);

  // Main loop setup
  ros::Rate loop_rate(100);

  while(ros::ok())
  {
    // Receive current state from robot
    if (!rrbot_hardware_interface.read(timestamp, period)) {
      ROS_FATAL_NAMED("rrbot_hardware_interface", "Failed to read state from robot. Shutting down!");
      ros::shutdown();
    }

    ...
    // Update the controllers
    controller_manager.update(timestamp, period);

    // Send new setpoint to robot
    rrbot_hardware_interface.write(timestamp, period);

    loop_rate.sleep();
  }
  return 0;
}
```

Let's launch the RRBot hardware

```bash 
roslaunch rrbot_bringup rrbot_hw_and_states.launch --screen
```

... and read its states in another terminal:

```bash
rostopic echo /joint_states
```

## Implementing a controller for ros_control

Let's take a look at how a controller is structured.

```C++
class RRBotControllerArray :
public controller_interface::Controller<hardware_interface::PositionJointInterface>
{
public:
  bool init(hardware_interface::PositionJointInterface * hw, ros::NodeHandle & nh);

  void starting(const ros::Time & time);

  void update(const ros::Time & time, const ros::Duration & period);

protected:
  std::vector<std::string> joint_names_;
  std::vector<hardware_interface::JointHandle> joints_;
  realtime_tools::RealtimeBuffer<std::vector<double> > commands_buffer_;

  using ControllerCommandMsg = std_msgs::Float64MultiArray;

  ros::Subscriber command_subscriber_;
  void commandCB(const ControllerCommandMsg::ConstPtr & msg);

  using ControllerStateMsg = control_msgs::JointControllerState;
  using ControllerStatePublisher = realtime_tools::RealtimePublisher<ControllerStateMsg>;

  std::unique_ptr<ControllerStatePublisher> state_publisher_;
};
```

while in the cpp file... 

```C++
void RRBotControllerArray::starting(const ros::Time & /*time*/)
{
  // Start controller with current joint positions
  std::vector<double> & commands = *commands_buffer_.readFromRT();
  for(size_t i = 0; i < joint_names_.size(); ++i)
  {
    commands[i] = joints_[i].getPosition();
  }
}

void RRBotControllerArray::update(const ros::Time & /*time*/, const ros::Duration & /*period*/)
{
  std::vector<double> & commands = *commands_buffer_.readFromRT();
  for(size_t i = 0; i < joint_names_.size(); ++i) {
    joints_[i].setCommand(commands[i]);
  }

  if (state_publisher_ && state_publisher_->trylock()) {
    state_publisher_->msg_.header.stamp = ros::Time::now();
    state_publisher_->msg_.set_point = joints_[0].getPosition();

    state_publisher_->unlockAndPublish();
  }
}

void RRBotControllerArray::commandCB(const ControllerCommandMsg::ConstPtr & msg)
{
  commands_buffer_.writeFromNonRT(msg->data);
}
```

To make your controller available to `ros_control`, it needs to be exported as a `pluginlib` plugin. This is one line at the bottom of your `.cpp` file as such

```C++
#include "pluginlib/class_list_macros.hpp"

PLUGINLIB_EXPORT_CLASS(rrbot_controller::RRBotControllerArray, controller_interface::ControllerBase)
```

add a matching xml file called `rrbot_controller.xml` into you packages:

```XML
<library path="librrbot_controller_array">
  <class name="rrbot_controller/RRBotControllerArray"
         type="rrbot_controller::RRBotControllerArray" base_class_type="controller_interface::ControllerBase">
    <description>
      RRBotControllerArray ros_control controller.
    </description>
  </class>
</library>
```

and finally, export this file in the `package.xml`:
```XML
  <export>
    <controller_interface plugin="${prefix}/rrbot_controller.xml"/>
  </export>
```

(also install this file in the package share directory in `CMakeLists.txt`).

Bringup the RRBot hardware with the array-controller:

`roslaunch rrbot_bringup rrbot_with_rrbot_controller_array.launch`

Let's publish some commands to the controller!

```bash
rostopic pub /rrbot_controller/command std_msgs/Float64MultiArray "layout:
  dim:
  - label: ''
    size: 0
    stride: 0
  data_offset: 0
data:
- 0.5
- 0.5"
```

#### We can achieve this with a standard forward command controller ([ros_controllers](https://github.com/ros-controls/ros_controllers) package):

```roslaunch rrbot_bringup rrbot.launch```

```bash
rostopic pub /forward_position_controller/command std_msgs/Float64MultiArray "layout:
  dim:
  - label: ''
    size: 0
    stride: 0
  data_offset: 0
data:
- 0.5
- 0.5"
```

## Implementing a controller accepting custom commands in ros_control ("jog" controller)

Now let's use a different command message! We will use `JointJog` to replace `Float64MultiArray`. These messages can be used in a compatible way. For our use-case, the commands we put in the `data` field can now be placed in the `displacements` field.

Here's a side-by-side comparison of the messages:

<table border="0">
 <tr>
    <td  style="text-align: left;"><b>std_msgs/Float64MultiArray</b></td>
    <td  style="text-align: left;"><b>control_msgs/JointJog</b></td>
 </tr>
 <tr>
    <td  style="text-align: left;">
        <pre>
std_msgs/MultiArrayLayout layout
  std_msgs/MultiArrayDimension[] dim
    string label
    uint32 size
    uint32 stride
  uint32 data_offset
float64[] data
        </pre>
    </td>
    <td  style="text-align: left;">
        <pre>
std_msgs/Header header
string[] joint_names
float64[] displacements
float64[] velocities
float64 duration
        </pre>
     </td>
 </tr>
</table>

Changing the command type in the hpp file is as simple as changing the `using` type alias.

```C++
  class RRBotController :
  public controller_interface::Controller<hardware_interface::PositionJointInterface>
  {
  public:
    ...

  protected:
    ...

    using ControllerCommandMsg = control_msgs::JointJog;

    ...
  };
```

And update field names used it the subscriber in cpp file:

```C++
...

void RRBotController::commandCB(const ControllerCommandMsg::ConstPtr & msg)
{
  commands_buffer_.writeFromNonRT(msg->displacements);
}
```

Bringup the hardware using newly created controller:

`roslaunch rrbot_bringup rrbot_with_rrbot_controller.launch`

And send some commands to it:

```bash 
rostopic pub /rrbot_controller/command control_msgs/JointJog "header:
  seq: 0
  stamp: {secs: 0, nsecs: 0}
  frame_id: ''
joint_names: ['']
displacements: [0.5, 0.5]
velocities: [0]
duration: 0.0"
```

# ROS2 + ros\_control = ros2\_control

In ros2_control, the functionalities of a HAL and RobotHW are separated. 

The role of HAL is taken by the ResourceManager which can manage multiple Sensor, System or Actuator components. 
These components serve as self-contained hardware drivers, essentially equivalent to ros_control's RobotHW.

![](images/components_architecture.png)


For the following demos we'll use the ROS2 Foxy environment, make sure to run

Checking rrbot

```
ros2 launch rrbot_description view_robot.launch.py
```

![rrbot rviz](images/rrbot_gui.png)

## Implementing a system component in ros2_control

The integration of `ros2_control` is done through the `ros2_control` URDF tag

```xml
<ros2_control name="${name}" type="system">
      <hardware>
          <plugin>fake_components/GenericSystem</plugin>
      </hardware>
      <joint name="joint1">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
      <joint name="joint2">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
    </ros2_control>
```

which will pick our pre-supplied implementation of a system component that always "simulates" perfect execution.

Let's write one!

```C++
class RRBotHardwareInterface
  : public hardware_interface::BaseInterface<hardware_interface::SystemInterface>
{
public:
  hardware_interface::return_type configure(const hardware_interface::HardwareInfo & info) override;

  std::vector<hardware_interface::StateInterface> export_state_interfaces() override;

  std::vector<hardware_interface::CommandInterface> export_command_interfaces() override;

  hardware_interface::return_type start() override;

  hardware_interface::return_type stop() override;

  hardware_interface::return_type read() override;

  hardware_interface::return_type write() override;

private:
  std::vector<double> hw_commands_;
  std::vector<double> hw_states_;
};
```

while in the cpp file...

```C++
hardware_interface::return_type RRBotHardwareInterface::read()
{
  // read robot states from hardware, in this example print only
  RCLCPP_INFO(rclcpp::get_logger("RRBotHardwareInterface"), "Reading...");

  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_states_.size(); ++i){
    RCLCPP_INFO(
      rclcpp::get_logger("RRBotHardwareInterface"),
      "Got state %.2f for joint %d!", hw_states_[i], i);
  }

  return hardware_interface::return_type::OK;
}

hardware_interface::return_type RRBotHardwareInterface::write()
{
  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_commands_.size(); ++i){
    hw_states_[i] = hw_states_[i] + (hw_commands_[i] - hw_states_[i]) / 100.0;
  }

  return hardware_interface::return_type::OK;
}
```

The updated ros2_control tag will look like this:
```xml
<ros2_control name="${name}" type="system">
      <hardware> 
        <plugin>rrbot_hardware_interface/RRBotHardwareInterface</plugin>
      </hardware>
      <joint name="joint1">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
      <joint name="joint2">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
</ros2_control>
```

## Implementing an array-based controller in ros2_control

```C++
class RRBotControllerArray : public controller_interface::ControllerInterface
{
public:
  controller_interface::return_type init(const std::string & controller_name) override;

  controller_interface::InterfaceConfiguration command_interface_configuration() const override;

  controller_interface::InterfaceConfiguration state_interface_configuration() const override;

  CallbackReturn on_configure(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_activate(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_deactivate(const rclcpp_lifecycle::State & previous_state) override;

  controller_interface::return_type update() override;

protected:
  std::vector<std::string> joint_names_;
  std::string interface_name_;

  using ControllerCommandMsg = example_interfaces::msg::Float64MultiArray;

  rclcpp::Subscription<ControllerCommandMsg>::SharedPtr command_subscriber_ = nullptr;
  realtime_tools::RealtimeBuffer<std::shared_ptr<ControllerCommandMsg>> input_command_;

  using ControllerStateMsg = control_msgs::msg::JointControllerState;
  using ControllerStatePublisher = realtime_tools::RealtimePublisher<ControllerStateMsg>;

  rclcpp::Publisher<ControllerStateMsg>::SharedPtr s_publisher_;
  std::unique_ptr<ControllerStatePublisher> state_publisher_;
};
```

Pluginlib export in ROS2 uses the same definition in the cpp file, but instead of exporting in `package.xml` we have to do this in the `CMakeLists.txt`:

```cmake
pluginlib_export_plugin_description_file(controller_interface rrbot_controller.xml)
```

Bring up ros2_control with the array-based controller running:
```
ros2 launch rrbot_bringup rrbot_with_rrbot_controller_array.launch.py
```

Let's test the controller by sending it some commands!

```
ros2 topic pub /rrbot_controller/commands example_interfaces/msg/Float64MultiArray "data:
- 0.5
- 0.5" 
```

## Implementing a "jog" controller in ros2_control

Let's replace Float64MultiArray with something more specific, such as JointJog. 

Modifying the code of the previous controller, simply change the `ControllerCommandMsg` type in the header

```C++
class RRBotController : public controller_interface::ControllerInterface
{
public:
  ...
protected:
  ...
  
  using ControllerCommandMsg = control_msgs::msg::JointJog;

  ...
};
```

...and re-adjust some of the implementation reference from `data` to `displacement`.

Testing the new controller: add controller and configuration file to launch file

`ros2 launch rrbot_bringup rrbot_with_rrbot_controller.launch.py`

Check which controllers are loaded now and which interfaces are claimed

```
ros2 control list_controllers
```

```
ros2 control list_hardware_interfaces
```

Examine the state of the controller on the state topic or the joint states topic

```
ros2 topic echo /rrbot_controller/state
```


```
ros2 topic echo /joint_states
```

```
ros2 topic pub /rrbot_controller/commands control_msgs/msg/JointJog "joint_names:
- joint1
- joint2
displacements:
- 0.5
- 0.5"
```

## [Concluding remarks]


[Write here your conclusions]

# References

* [ros_control paper in the Journal of Open Source Software](https://joss.theoj.org/papers/10.21105/joss.00456)
* ros_control
 * http://wiki.ros.org/ros_control
 * http://wiki.ros.org/ros_controllers (select individual controllers) 
 * https://github.com/ros-controls/roadmap/blob/master /documentation_resources.md
* ros2_control
 * https://ros-controls.github.io/control.ros.org/
 * https://github.com/ros-controls/ros2_control
 * https://github.com/ros-controls/ros2_controllers
 * https://github.com/ros-controls/ros2_control_demos

# Extra ROSject Documentation

All source code for ros_control on ROS Noetic can be found in `catkin_ws` while all resources for ros2_control on ROS Foxy is located in `ros2_ws`.